--- Day 9: Rope Bridge ---
This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

```
..##..
...##.
.####.
....#.
s###..
```

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [1]:
import sys


In [3]:


with open("d9.txt") as f:
    instructions = [x.split() for x in f]

In [4]:
# n_knots = 2 # Part 1
n_knots = 10  # Part 2
pos_h = (0,0)
pos_ts     = [(0,0)   for _ in range(n_knots-1)]
history_ts = [{(0,0)} for _ in range(n_knots-1)]

sign = lambda x: (1, -1)[x < 0]

for k, t in instructions:
    d = {"R": (1,0), "L": (-1,0), "U": (0,1), "D": (0,-1)}[k]
    for _ in range(int(t)):
        pos_h = pos_h[0]+d[0], pos_h[1]+d[1]
        pos_pred = pos_h
        for i, history_t in enumerate(history_ts):
            x, y = pos_ts[i][0], pos_ts[i][1]
            dx, dy = x-pos_pred[0], y-pos_pred[1]
            
            if (abs(dx) > 0 and abs(dy) > 1) or (abs(dx) > 1 and abs(dy) > 0):
                pos_ts[i] = x-sign(dx), y-sign(dy)
            elif abs(dx) > 1:
                pos_ts[i] = x-sign(dx), y
            elif abs(dy) > 1:
                pos_ts[i] = x, y-sign(dy)
    
            pos_pred = pos_ts[i]
            history_t.add(pos_ts[i])

print(len(history_ts[-1]))

2545


In [1]:
import time
from numpy import sign

In [2]:
def moverope(moves, ropelen):
    ropesegs = [[0, 0] for _ in range(ropelen)]
    visited = set()
    for move in moves:
        dir, movesteps = move.split()
        for i in range(0, int(movesteps)):
            ropesegs[0][0] += {"U": 1, "D": -1}.get(dir, 0)
            ropesegs[0][1] += {"R": 1, "L": -1}.get(dir, 0)
            for i in range(1, len(ropesegs)):
                ydist = ropesegs[i-1][0]-ropesegs[i][0]
                xdist = ropesegs[i-1][1]-ropesegs[i][1]
                if ydist**2 + xdist**2 > 2: 
                    ropesegs[i][0] += sign(ydist)
                    ropesegs[i][1] += sign(xdist)
            visited.add(str(ropesegs[-1]))
    return len(visited)

In [3]:
def run():
    moves = open('d9.txt', 'r').read().rstrip().split("\n")
    print("\nDay 9: Rope Bridge")
    print("pt 1: ", moverope(moves, 2) )
    print("pt 2: ", moverope(moves, 10) )


In [4]:
if __name__ == "__main__":
    start = time.time()
    run()
    end = time.time()
    print("elapsed time: %fms\n" % ((end-start)*1000))


Day 9: Rope Bridge
pt 1:  6030
pt 2:  2545
elapsed time: 134.802103ms

